In [10]:
import pandas as pd
import numpy as np

In [11]:
df_household = pd.read_excel('household_bmc.xlsx')
df_vehicle = pd.read_excel('vehicle_bmc.xlsx')
df_person = pd.read_excel('person_bmc.xlsx')
df_trip = pd.read_excel('trip_bmc.xlsx')

In [12]:
df_household.replace(-9, np.nan, inplace=True)
df_vehicle.replace(-9, np.nan, inplace=True)
df_person.replace(-9, np.nan, inplace=True)
df_trip.replace(-9, np.nan, inplace=True)

In [13]:
print(df_household)
print(df_vehicle)
print(df_person)
print(df_trip)

      household_id  in_tpb  in_bmc  home_state_fips home_state_puma  \
0        140000006       0       0               24        24_00100   
1        140000013       0       0               24        24_00100   
2        140000017       0       0               24        24_00100   
3        140000019       0       0               24        24_00100   
4        140000024       0       0               24        24_00100   
...            ...     ...     ...              ...             ...   
8308     180256058       1       1               24        24_01201   
8309     180256271       1       1               24        24_00902   
8310     180256360       1       1               24        24_01204   
8311     180256797       1       1               24        24_00902   
8312     180256875       1       1               24        24_01203   

      area_type  home_state_county_fips  home_tract_fips  home_tract_fips20  \
0            10                   24001              100            

In [29]:
df_consolidated = df_person.copy()
df_trip["staying_time"] = 0
max_trips = df_trip['tripno'].max()

# Create an empty list to hold the DataFrames for concatenation
dfs_to_concat = [df_consolidated]
# Iterate over the columns of df_trip
for col in df_trip.columns:
    if col not in ['household_id', 'person_id', 'trip_id', 'persno']:
        # Create a DataFrame with columns named col_i, where i is the trip number
        temp_df = pd.DataFrame(0, index=df_person.index, columns=[f'{col}_{i}' for i in range(1, max_trips + 1)])
        dfs_to_concat.append(temp_df)

# Concatenate all the DataFrames in the list along the axis 1 (columns)
df_consolidated = pd.concat(dfs_to_concat, axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
17853    0
17854    0
17855    0
17856    0
17857    0
Name: tripno_22, Length: 17858, dtype: int64


In [18]:
    data = {'col1': [1, 7, 3], 'col2': [4, 5, 6]}
    df = pd.DataFrame(data)
    
    def my_function(row):
        if row['col1'] > row['col2']:
            return 'greater'
        else:
            return 'less_or_equal'
    
    df['new_col'] = df.apply(my_function, axis=1)
    print(df)

   col1  col2        new_col
0     1     4  less_or_equal
1     7     5        greater
2     3     6  less_or_equal
